# **DATA CLEANING**

In [47]:
# Import library
import numpy as np
import pandas as pd

In [48]:
# Import dan preview dataset
df = pd.read_csv('portugal_listinigs.csv')
df.head()

/tmp/ipython-input-1040222966.py:2: DtypeWarning: Columns (9,10,12,13,14,15,16,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('portugal_listinigs.csv')


,Price,District,City,Town,Type,EnergyCertificate,GrossArea,TotalArea,Parking,HasParking,...,Elevator,ElectricCarsCharging,TotalRooms,NumberOfBedrooms,NumberOfWC,ConservationStatus,LivingArea,LotSize,BuiltArea,NumberOfBathrooms
0,"780,000",Vila Real,Valpaços,Carrazedo de Montenegro e Curros,Farm,NC,200,"552,450",0,False,...,False,NaN,NaN,NaN,NaN,NaN,120,NaN,NaN,0
1,"223,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,NaN,81,1,True,...,True,NaN,2,NaN,NaN,NaN,81,NaN,NaN,2
2,"228,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,NaN,108,1,True,...,True,NaN,2,NaN,NaN,NaN,108,NaN,NaN,2
3,"250,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,NaN,114,1,True,...,True,NaN,2,NaN,NaN,NaN,114,NaN,NaN,0
4,"250,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,NaN,114,1,True,...,True,NaN,2,NaN,NaN,NaN,114,NaN,NaN,2


In [49]:
# Mengecek informasi dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135536 entries, 0 to 135535
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Price                  135236 non-null  float64
 1   District               135536 non-null  object 
 2   City                   135536 non-null  object 
 3   Town                   135534 non-null  object 
 4   Type                   135520 non-null  object 
 5   EnergyCertificate      135522 non-null  object 
 6   GrossArea              27638 non-null   float64
 7   TotalArea              127153 non-null  float64
 8   Parking                135342 non-null  float64
 9   HasParking             68215 non-null   object 
 10  Floor                  27929 non-null   object 
 11  ConstructionYear       88021 non-null   float64
 12  EnergyEfficiencyLevel  67289 non-null   object 
 13  PublishDate            29239 non-null   object 
 14  Garage                 67289 non-nul

In [50]:
# Mengecek informasi statistik data yang bertipe float
df.describe()

,Price,GrossArea,TotalArea,Parking,ConstructionYear,TotalRooms,NumberOfBedrooms,NumberOfWC,LivingArea,LotSize,BuiltArea,NumberOfBathrooms
count,"135,236","27,638","127,153","135,342","88,021","73,244","47,041","57,256","104,952","39,583","26,617","128,700"
mean,"368,137","2,790","514,078",1,"1,989",3,3,0,"1,403","56,369","3,087",2
std,"3,804,044","115,367","172,293,714",1,27,11,2,1,"34,746","5,208,474","118,514",2
min,1,-7,"-7,196,067",0,"1,900",0,0,-15,0,0,-1,-13
25%,"84,000",100,91,0,"1,973",2,2,0,80,258,105,0
50%,"210,000",164,159,0,"1,994",3,3,0,118,679,168,1
75%,"395,000",294,500,1,"2,009",4,3,1,204,"2,890",300,2
max,"1,380,000,000","12,750,000","61,420,071,105",3,"2,025","2,751",21,59,"5,429,000","992,301,000","12,750,000",131


##  **A. DROP Redundant Kolom**
---


*   Menghapus kolom-kolom yang memiliki redundansi informasi karena dua kolom atau lebih menyimpan data yang sama
*   Pengecekan dilakukan dengan menganalisis kolom yang memiliki nama kolom yang mirip





###   1.  Kolom **'Parking'** dan **'HasParking'**







In [51]:
# Cek missing value
parking_missing = df['Parking'].isnull().sum()
hasparking_missing = df['HasParking'].isnull().sum()

print(f"Parking missing    : {parking_missing:,} ({parking_missing/len(df)*100:.2f}%)")
print(f"HasParking missing : {hasparking_missing:,} ({hasparking_missing/len(df)*100:.2f}%)")

Parking missing    : 194 (0.14%)
HasParking missing : 67,321 (49.67%)


In [52]:
# Cek korelasi
irisan_value = df[(df['Parking'].notnull()) & (df['HasParking'].notnull())].copy()
print(f"Jumlah irisan row Parking & HasParking : {len(irisan_value):,}")

if len(irisan_value) > 0:

    # Convert HasParking ke boolean
    irisan_value['HasParking_bool'] = irisan_value['HasParking'].map({
        'True': True, 'False': False, True: True, False: False})

    # Mengecek apakah parking > 0 sama dengan HasParking = True dengan mengubah kolom 'Parking' menjadi tipe boolean
    irisan_value['Parking_bool'] = irisan_value['Parking'] > 0 # bernilai true apabila parking > 0 sama dengan HasParking = True

    # Membandingkan nilai yang sama di kedua kolom
    match_count = (irisan_value['Parking_bool'] == irisan_value['HasParking_bool']).sum()
    mismatch_count = (irisan_value['Parking_bool'] != irisan_value['HasParking_bool']).sum()

    print(f"  Matching : {match_count:,} ({match_count/len(irisan_value)*100:.1f}%)")
    print(f"  Mismatching: {mismatch_count:,} ({mismatch_count/len(irisan_value)*100:.1f}%)")

Jumlah irisan row Parking & HasParking : 68,053
  Matching : 68,053 (100.0%)
  Mismatching: 0 (0.0%)


**Kesimpulan:**
1.   Kolom **'Parking'** dan **'HasParking'** 100.0% matching
2.   Karena kolom **'Parking'** memiliki nilai missing value yang lebih rendah, maka:
> - kolom **'Parking'** dipertahankan
> - kolom **'HasParking'** dihapus











In [53]:
# Menghapus kolom HasParking
df = df.drop(columns=['HasParking'])
df.columns

Index(['Price', 'District', 'City', 'Town', 'Type', 'EnergyCertificate',
       'GrossArea', 'TotalArea', 'Parking', 'Floor', 'ConstructionYear',
       'EnergyEfficiencyLevel', 'PublishDate', 'Garage', 'Elevator',
       'ElectricCarsCharging', 'TotalRooms', 'NumberOfBedrooms', 'NumberOfWC',
       'ConservationStatus', 'LivingArea', 'LotSize', 'BuiltArea',
       'NumberOfBathrooms'],
      dtype='object')

Kolom **'HasParking'** berhasil dihapus

###   2.       Kolom **'EnergyCertificate'** dan **'EnergyEfficiencyLevel'**

In [54]:
# Cek missing value
EnergyCertificate_missing = df['EnergyCertificate'].isnull().sum()
EnergyEfficiencyLevel_missing = df['EnergyEfficiencyLevel'].isnull().sum()

print(f"EnergyCertificate missing    : {EnergyCertificate_missing:,} ({EnergyCertificate_missing/len(df)*100:.2f}%)")
print(f"EnergyEfficiencyLevel missing : {EnergyEfficiencyLevel_missing:,} ({EnergyEfficiencyLevel_missing/len(df)*100:.2f}%)")

EnergyCertificate missing    : 14 (0.01%)
EnergyEfficiencyLevel missing : 68,247 (50.35%)


In [55]:
# Cek korelasi
irisan_value = df[(df['EnergyCertificate'].notnull()) & (df['EnergyEfficiencyLevel'].notnull())].copy()
print(f"Jumlah irisan row EnergyCertificate & EnergyEfficiencyLevel : {len(irisan_value):,}")

if len(irisan_value) > 0:

  # Mengecek apakah nilai di kedua kolom sama
  match_count = (irisan_value['EnergyCertificate'] == irisan_value['EnergyEfficiencyLevel']).sum()
  mismatch_count = (irisan_value['EnergyCertificate'] != irisan_value['EnergyEfficiencyLevel']).sum()

  print(f"  Matching : {match_count:,} ({match_count/len(irisan_value)*100:.1f}%)")
  print(f"  Mismatching : {mismatch_count:,} ({mismatch_count/len(irisan_value)*100:.1f}%)")

Jumlah irisan row EnergyCertificate & EnergyEfficiencyLevel : 67,289
  Matching : 67,289 (100.0%)
  Mismatching : 0 (0.0%)


**Kesimpulan:**
1.   Kolom **'EnergyCertificate'** dan **'EnergyEfficiencyLevel'** 100.0% matching
2.   Karena kolom **'EnergyCertificate'** memiliki nilai missing value yang lebih rendah, maka:
> - kolom **'EnergyCertificate'** dipertahankan
> - kolom **'EnergyEfficiencyLevel'** dihapus


In [56]:
# Menghapus kolom EnergyEfficiencyLevel
df = df.drop(columns=['EnergyEfficiencyLevel'])
df.columns

Index(['Price', 'District', 'City', 'Town', 'Type', 'EnergyCertificate',
       'GrossArea', 'TotalArea', 'Parking', 'Floor', 'ConstructionYear',
       'PublishDate', 'Garage', 'Elevator', 'ElectricCarsCharging',
       'TotalRooms', 'NumberOfBedrooms', 'NumberOfWC', 'ConservationStatus',
       'LivingArea', 'LotSize', 'BuiltArea', 'NumberOfBathrooms'],
      dtype='object')

Kolom **'EnergyEfficiencyLevel'** berhasil dihapus

##  **B. DROP Kolom yang Tidak Relevan**
---



In [57]:
df.drop(columns=['NumberOfWC', 'ElectricCarsCharging'], inplace=True)
df.columns

Index(['Price', 'District', 'City', 'Town', 'Type', 'EnergyCertificate',
       'GrossArea', 'TotalArea', 'Parking', 'Floor', 'ConstructionYear',
       'PublishDate', 'Garage', 'Elevator', 'TotalRooms', 'NumberOfBedrooms',
       'ConservationStatus', 'LivingArea', 'LotSize', 'BuiltArea',
       'NumberOfBathrooms'],
      dtype='object')

Alasan drop kolom:
*  **NumberOfWC (57,9% missing)**: Ada alternatif (NumberOfBathrooms)
*   **ElectricCarsCharging (50.4% missing)**: Tidak relevan (terlalu niche)

##  **C. Cleaning Duplicate Value**
---



In [58]:
# Cek jumlah duplikat
jumlah_duplikat = df.duplicated().sum()
print(f"Jumlah baris duplikat: {jumlah_duplikat}")

# Hapus duplikat
df = df.drop_duplicates()
print(f"Setelah hapus duplikat: {len(df)} baris")

Jumlah baris duplikat: 8924
Setelah hapus duplikat: 126612 baris


##  **D. Cleaning Missing Value**
---


Menghapus kolom-kolom yang memiliki missing value

In [59]:
missing_analysis = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100),
})
missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
ConservationStatus,108565,86
BuiltArea,101780,80
GrossArea,100598,79
Floor,100513,79
PublishDate,98697,78
LotSize,89711,71
NumberOfBedrooms,82800,65
Garage,63912,50
TotalRooms,57808,46
ConstructionYear,43313,34


### 1. Cleaning Kolom dengan Missing_Percentage < 5%

In [60]:
# Drop row yang missing value
df = df.dropna(subset=['Price', 'Parking', 'Elevator', 'Type', 'EnergyCertificate', 'Town'])

In [61]:
missing_analysis = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100),
})
missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
ConservationStatus,108178,86
BuiltArea,101389,80
GrossArea,100256,79
Floor,100173,79
PublishDate,98278,78
LotSize,89357,71
NumberOfBedrooms,82492,65
Garage,63627,50
TotalRooms,57664,46
ConstructionYear,43112,34


### 2. Cleaning Kolom dengan Missing_Percentage > 70%

In [62]:
# Drop kolom yang memiliki missing value tinggi
df.drop(columns=['ConservationStatus', 'BuiltArea', 'GrossArea', 'Floor', 'PublishDate', 'LotSize'], inplace=True)
df.columns

Index(['Price', 'District', 'City', 'Town', 'Type', 'EnergyCertificate',
       'TotalArea', 'Parking', 'ConstructionYear', 'Garage', 'Elevator',
       'TotalRooms', 'NumberOfBedrooms', 'LivingArea', 'NumberOfBathrooms'],
      dtype='object')

Alasan drop kolom:


*   **ConservationStatus (85.7% missing)** : Terlalu sparse
*   **BuiltArea (80.3% missing)** : Ada alternatif (LivingArea)
*   **GrossArea (79.5% missing**) : Terlalu sparse
*   **Floor (79.4% missing)** : Terlalu sparse
*   **PublishDate (77.9% missing)**: Terlalu sparse
*   **LotSize (70.8% missing)** : Hanya relevant untuk tipe tertentu












In [63]:
# Mengecek missing value setelah cleaning
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_analysis = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Percentage': missing_percentage
})

missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
NumberOfBedrooms,82492,65
Garage,63627,50
TotalRooms,57664,46
ConstructionYear,43112,34
LivingArea,28433,23
TotalArea,7772,6
NumberOfBathrooms,6448,5
EnergyCertificate,0,0
Type,0,0
Town,0,0


### 3. Cleaning Kolom dengan Missing_Percentage 5%-6% **(TotalArea & NumberOfBathrooms)**

In [64]:
# Mengisi missing value dengan nilai median
## TotalArea
df['TotalArea'] = df.groupby('Type')['TotalArea'].transform(lambda x: x.fillna(x.median()))

## NumberOfBathrooms
df['NumberOfBathrooms'] =  df.groupby('Type')['NumberOfBathrooms'].transform(lambda x: x.fillna(x.median()))

In [65]:
# Mengecek missing value setelah cleaning
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_analysis = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Percentage': missing_percentage
})

missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
NumberOfBedrooms,82492,65
Garage,63627,50
TotalRooms,57664,46
ConstructionYear,43112,34
LivingArea,28433,23
District,0,0
Price,0,0
TotalArea,0,0
EnergyCertificate,0,0
Type,0,0


Berhasil menghapus missing value pada kolom **(TotalArea & NumberOfBathrooms)**

In [66]:
# Mengecek informasi statistik kolom NumberOfBathrooms dan TotalArea
df[['TotalArea', 'NumberOfBathrooms']].describe()

,TotalArea,NumberOfBathrooms
count,"126,180","126,180"
mean,"517,820",1
std,"172,956,734",2
min,"-7,196,067",-13
25%,94,0
50%,176,1
75%,803,2
max,"61,420,071,105",131


Nilai **Min** kolom **'NumberOfBathrooms'** dan **'TotalArea'** bernilai **NEGATIF**

In [67]:
# Untuk nilai NEGATIF
## Bathroom
df.loc[df['NumberOfBathrooms'] < 0, 'NumberOfBathrooms'] = np.nan

## TotalArea
df.loc[df['TotalArea'] <= 0, 'TotalArea'] = np.nan

# Mengecek describe data
df[['TotalArea', 'NumberOfBathrooms']].describe()

,TotalArea,NumberOfBathrooms
count,"125,282","126,177"
mean,"521,589",1
std,"173,575,487",2
min,1,0
25%,95,0
50%,179,1
75%,828,2
max,"61,420,071,105",131


Row yang bernilai **NEGATIF** pada kolom **'NumberOfBathrooms'** dan **'TotalArea'** berhasil dihapus

#### Handling kolom **'Total Area'**

In [68]:
# Menandai nilai TotalArea yang tidak masuk akal (< 2m2) sebagai NaN
df.loc[df['TotalArea'] < 2, 'TotalArea'] = np.nan

# Tipe hunian yang TIDAK BOLEH < 15m2
tipe_hunian_utama = ['Apartment', 'House', 'Studio', 'Duplex', 'Mansion', 'Estate', 'Manor']
mask_hunian_mini = (df['Type'].isin(tipe_hunian_utama)) & (df['TotalArea'] < 15)
df.loc[mask_hunian_mini, 'TotalArea'] = np.nan

# Tipe lahan TIDAK BOLEH < 50m2
tipe_lahan = ['Land', 'Farm', 'Industrial', 'Investment']
mask_lahan_mini = (df['Type'].isin(tipe_lahan)) & (df['TotalArea'] < 50)
df.loc[mask_lahan_mini, 'TotalArea'] = np.nan

# Menandai row yang tidak logis (LivingArea > TotalArea) sebagai NaN
mask_unlogic = df['LivingArea'] > df['TotalArea']
df.loc[mask_unlogic, ['LivingArea', 'TotalArea']] = np.nan

# Total Area TIDAK BOLEH > 10 Juta m2
df.loc[df['TotalArea'] > 10_000_000, 'TotalArea'] = np.nan

# Tipe yang TIDAK BOLEH > 5.000 m2
mask_apt = (df['Type'].isin(['Apartment', 'Studio', 'Duplex', 'Store'])) & (df['TotalArea'] > 5000)
df.loc[mask_apt, 'TotalArea'] = np.nan

# Tipe yang TIDAK BOLEH >= 100.000 m2
mask_house = (df['Type'].isin(['House', 'Manor', 'Building', 'Office'])) & (df['TotalArea'] >= 100000)
df.loc[mask_house, 'TotalArea'] = np.nan

# Mengisi semua NaN dengan median
df['TotalArea'] = df.groupby('Type')['TotalArea'].transform(lambda x: x.fillna(x.median()))

#### Handling kolom **'NumberOfBathrooms'**

In [69]:
# Menampilkan semua kolom untuk baris yang NumberOfBathrooms nya bernilai 131 (karena selisih Q3 dan Max sangat jauh)
df_131 = df[df['NumberOfBathrooms'] == 131]
display(df_131)

,Price,District,City,Town,Type,EnergyCertificate,TotalArea,Parking,ConstructionYear,Garage,Elevator,TotalRooms,NumberOfBedrooms,LivingArea,NumberOfBathrooms
135169,"55,000",Portalegre,Alter do Chão,Cunheira,House,NC,233,0,"1,970",False,False,2,1,NaN,131


**DROP ROW** dengan NumberOfBathrooms = 131 karena anomali data:
*   **Tipe & Kapasitas:** Tidak logis sebuah 'House' dengan 2 ruangan memiliki 131 kamar mandi
*   **Dimensi:** TotalArea hanya 41 m² dan tidak memungkinkan menampung 131 kamar mandi
*   **Inkonsistensi:** LivingArea > TotalArea





In [70]:
# Drop row dengan NumberOfBathrooms = 131
df = df.drop(df[df['NumberOfBathrooms'] == 131].index)

In [71]:
df['NumberOfBathrooms'].describe()

,NumberOfBathrooms
count,"126,176"
mean,1
std,2
min,0
25%,0
50%,1
75%,2
max,90


ROW dengan NumberOfBathrooms = 131 **berhasil dihapus**

In [72]:
# Type yang wajib mempunyai Bathroom
tipe_hunian = ['House', 'Apartment', 'Store', 'Building', 'Office', 'Hotel', 'Estate', 'Duplex', 'Mansion', 'Manor', 'Studio', 'Industrial']

# Mengubah nilai 0 menjadi NaN khusus untuk tipe yang masuk dalam tipe hunian
mask_hunian_nol = (df['Type'].isin(tipe_hunian)) & (df['NumberOfBathrooms'] == 0)
df.loc[mask_hunian_nol, 'NumberOfBathrooms'] = np.nan

# Mengisi semua NaN dengan nilai median
median_bath = df[df['NumberOfBathrooms'] > 0]['NumberOfBathrooms'].median()
df['NumberOfBathrooms'] = df.groupby('Type')['NumberOfBathrooms'].transform(lambda x: x.fillna(x.median()))

### 4. Cleaning Kolom dengan Missing_Percentage 20%-65% **(NumberOfBedrooms,  Garage, TotalRooms, LivingArea)**

Kolom **ConstructionYear** KEEP AS IS, dengan alasan:
*   Merupakan data temporal
*   Tidak menciptakan pola palsu
*   Bisa menyebabkan misinterpretasi





In [73]:
# Mengisi missing value dengan nilai median
## NumberOfBedrooms
df['NumberOfBedrooms'] = df.groupby('Type')['NumberOfBedrooms'].transform(lambda x: x.fillna(x.median()))

## TotalRooms
df['TotalRooms'] = df.groupby('Type')['TotalRooms'].transform(lambda x: x.fillna(x.median()))

## LivingArea
df['LivingArea'] = df.groupby('Type')['LivingArea'].transform(lambda x: x.fillna(x.median()))

In [74]:
# Fungsi untuk mendapatkan nilai modus
def fill_with_mode(x):
    mode_val = x.mode()
    if not mode_val.empty:
        return x.fillna(mode_val[0])
    return x

# Mengisi missing value Garage berdasarkan modus
df['Garage'] = df.groupby('Type')['Garage'].transform(fill_with_mode)

/tmp/ipython-input-1762460744.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return x.fillna(mode_val[0])


In [75]:
# Mengecek missing value setelah cleaning
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_analysis = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Percentage': missing_percentage
})

missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
ConstructionYear,43112,34
District,0,0
Price,0,0
Town,0,0
Type,0,0
EnergyCertificate,0,0
City,0,0
TotalArea,0,0
Parking,0,0
Garage,0,0


Berhasil menghapus missing value pada kolom **(NumberOfBedrooms,  Garage, TotalRooms, ConstructionYear, LivingArea)**

In [76]:
# Mengecek informasi statistik kolom bertipe float
df[['NumberOfBedrooms', 'TotalRooms', 'ConstructionYear', 'LivingArea']].describe()

,NumberOfBedrooms,TotalRooms,ConstructionYear,LivingArea
count,"126,179","126,179","83,067","126,179"
mean,2,2,"1,989","1,180"
std,2,8,27,"24,579"
min,0,0,"1,900",0
25%,0,0,"1,972",90
50%,2,3,"1,993",142
75%,3,4,"2,008",408
max,21,"2,751","2,025","5,429,000"


#### Handling kolom **'NumberOfBedrooms'**

In [77]:
# Type hunian yang HARUS punya kamar
tipe_hunian_bed = ['House', 'Apartment', 'Hotel', 'Mansion', 'Estate', 'Duplex']
mask_hunian_nol = (df['Type'].isin(tipe_hunian_bed)) & (df['NumberOfBedrooms'] <= 0)
df.loc[mask_hunian_nol, 'NumberOfBedrooms'] = np.nan

# Type yang HARUS 0 kamar
must_be_zero = ['Land', 'Warehouse', 'Store', 'Industrial', 'Garage', 'Other - Commercial']
df.loc[df['Type'].isin(must_be_zero), 'NumberOfBedrooms'] = 0

# Kamar tidak boleh melebihi total rooms
mask_kamar_anomali = df['NumberOfBedrooms'] > df['TotalRooms']
df.loc[mask_kamar_anomali, 'NumberOfBedrooms'] = np.nan

# Mengisi NaN dengan Median
df['NumberOfBedrooms'] = df['NumberOfBedrooms'].fillna(df.groupby('Type')['NumberOfBedrooms'].transform('median'))


#### Handling kolom **'TotalRooms'**

In [78]:
# Total Room HARUS 0
rooms_must_zero = ['Land', 'Garage']
df.loc[df['Type'].isin(rooms_must_zero), 'TotalRooms'] = 0

# Total Room TIDAK BOLEH 0
mask_error_nol = (~df['Type'].isin(rooms_must_zero)) & (df['TotalRooms'] < 1)
df.loc[mask_error_nol, 'TotalRooms'] = np.nan

# Total Room TIDAK BOLEH <= NumberOfBedrooms atau NumberOfBathrooms
mask_logic_rooms = (~df['Type'].isin(rooms_must_zero)) & \
                   ((df['TotalRooms'] <= df['NumberOfBedrooms']) | \
                    (df['TotalRooms'] <= df['NumberOfBathrooms']))
df.loc[mask_logic_rooms, 'TotalRooms'] = np.nan

# Mengisi NaN dengan median
df['TotalRooms'] = df['TotalRooms'].fillna(df.groupby('Type')['TotalRooms'].transform('median'))

#### Handling kolom **'LivingArea'**

In [79]:
# Living area TIDAK BOLEH <= Total area
mask_logic_error = df['LivingArea'] > df['TotalArea']
df.loc[mask_logic_error, 'LivingArea'] = np.nan

# Type Land harus 0 karena hanya tanah
zero_living_types = ['Land']
df.loc[df['Type'].isin(zero_living_types), 'LivingArea'] = 0

# Type Hunian TIDAK BOLEH < 10m2
tipe_hunian = ['House', 'Apartment', 'Duplex']
mask_terlalu_kecil = (df['Type'].isin(tipe_hunian)) & (df['LivingArea'] < 10)
df.loc[mask_terlalu_kecil, 'LivingArea'] = np.nan

# Tipe yang luas bangunannya biasanya kecil
small_living_types = ['Garage', 'Storage']
mask_small_excess = (df['Type'].isin(small_living_types)) & (df['LivingArea'] > 100)
df.loc[mask_small_excess, 'LivingArea'] = np.nan

# Tipe bangunan besar
tipe_bangunan_besar = ['Estate', 'Mansion', 'Hotel']
mask_area_ekstrem = (df['LivingArea'] > 5000) & (~df['Type'].isin(tipe_bangunan_besar))
df.loc[mask_area_ekstrem, 'LivingArea'] = np.nan

# Mengisi NaN dengan Median
df['LivingArea'] = df['LivingArea'].fillna(df.groupby('Type')['LivingArea'].transform('median'))



In [80]:
# Memastikan kalau missing value sudah hilang
missing_count = df.isnull().sum()
missing_percentage = (df.isnull().sum() / len(df)) * 100

missing_analysis = pd.DataFrame({
    'Missing_Count': missing_count,
    'Missing_Percentage': missing_percentage
})

missing_analysis.sort_values('Missing_Percentage', ascending=False)

,Missing_Count,Missing_Percentage
ConstructionYear,43112,34
District,0,0
Price,0,0
Town,0,0
Type,0,0
EnergyCertificate,0,0
City,0,0
TotalArea,0,0
Parking,0,0
Garage,0,0


##  **E. Standardisasi**


###  EnergyCertificate (No Certificate → NC)


In [81]:
# Mengubah nilai No Certificate → NC
df['EnergyCertificate'] = df['EnergyCertificate'].replace({'No Certificate': 'NC'})

##  **F. Handle Outlier Price**

In [82]:
df['Price'].describe()

,Price
count,"126,179"
mean,"370,622"
std,"3,936,246"
min,1
25%,"85,000"
50%,"210,000"
75%,"390,000"
max,"1,380,000,000"


Memiliki masalah outlier yang sangat ekstrem karena rentang datanya terlalu jauh

In [83]:
# Menghapus harga yang terlalu rendah (<10000)
df = df[df['Price'] >= 10000]

# Menggunakan IQR untuk handling outlier
Q1 = df['Price'].quantile(0.25)
Q3 = df['Price'].quantile(0.75)
IQR = Q3 - Q1

# Menentukan batas atas
upper_bound = Q3 + 3 * IQR # agar tidak banyak data yang hilang
df = df[df['Price'] <= upper_bound]

In [84]:
df['Price'].describe()

,Price
count,"118,682"
mean,"274,327"
std,"250,925"
min,"10,000"
25%,"85,000"
50%,"200,000"
75%,"365,000"
max,"1,310,000"


##  **G. Ubah Tipe Data**

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 118682 entries, 0 to 135535
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Price              118682 non-null  float64
 1   District           118682 non-null  object 
 2   City               118682 non-null  object 
 3   Town               118682 non-null  object 
 4   Type               118682 non-null  object 
 5   EnergyCertificate  118682 non-null  object 
 6   TotalArea          118682 non-null  float64
 7   Parking            118682 non-null  float64
 8   ConstructionYear   78643 non-null   float64
 9   Garage             118682 non-null  bool   
 10  Elevator           118682 non-null  object 
 11  TotalRooms         118682 non-null  float64
 12  NumberOfBedrooms   118682 non-null  float64
 13  LivingArea         118682 non-null  float64
 14  NumberOfBathrooms  118682 non-null  float64
dtypes: bool(1), float64(8), object(6)
memory usage: 13.7+ MB

In [86]:
# konversi tipe data ke category
kolom_kategori = ['District', 'City', 'Town', 'Type', 'EnergyCertificate']
df[kolom_kategori] = df[kolom_kategori].astype('category')

# konversi tipe data ke int
kolom_integer = ['Parking', 'TotalRooms', 'NumberOfBedrooms', 'NumberOfBathrooms']
df[kolom_integer] = df[kolom_integer].astype('int')

# konversi ke tipe boolean
kolom_boolean = ['Elevator', 'Garage']
df[kolom_boolean] = df[kolom_boolean].astype('bool')



---



In [87]:
# Mengatur format tampilan angka float
pd.options.display.float_format = '{:,.0f}'.format

In [88]:
# Menampilkan preview df
df.head()

,Price,District,City,Town,Type,EnergyCertificate,TotalArea,Parking,ConstructionYear,Garage,Elevator,TotalRooms,NumberOfBedrooms,LivingArea,NumberOfBathrooms
0,"780,000",Vila Real,Valpaços,Carrazedo de Montenegro e Curros,Farm,NC,"552,450",0,NaN,False,False,4,3,120,0
1,"223,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,81,1,NaN,False,True,3,2,81,2
2,"228,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,108,1,NaN,False,True,3,2,108,2
3,"250,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,114,1,NaN,False,True,3,2,114,2
4,"250,000",Faro,São Brás de Alportel,São Brás de Alportel,Apartment,A+,114,1,NaN,False,True,3,2,114,2


In [90]:
# Mengetahui rows dan kolom df
df.shape

(118682, 15)

In [46]:
# Mengeksport CSV
df.to_csv('data clean_Zafirah Aida Adista.csv', index=False)